In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
import xgboost as xgb
from sklearn.model_selection import cross_val_score, train_test_split
import pandas as pd

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [40]:
df = pd.read_csv('competition_table.csv',sep=';',index_col=0)
# replace commas for dots and convert previous string column into float
def processCol(col):
    try:
        return col.astype(str).apply(lambda val: val.replace(',','.')).astype(float)
    except:
        return col


df = df.apply(processCol)
print(len(df))
keep_seasons = ['2019-2020','2020-2021','2021-2022']
df = df[df.season.isin(keep_seasons)]

print(len(df))
df.head()

64166
18572


,season,div,odds_home_team_win,odds_draw,odds_away_team_win,home_team_match_nr,home_team_goal_roll1_sum,home_team_goal_roll2_sum,home_team_goal_roll3_sum,home_team_goal_roll4_sum,...,odds_away_team_draw_roll4_mean,odds_away_team_defeat_roll1_mean,odds_away_team_defeat_roll2_mean,odds_away_team_defeat_roll3_mean,odds_away_team_defeat_roll4_mean,away_team_point_mean,away_team_expected_point_mean,home_win_flag,draw_flag,away_win_flag
match_id,,,,,,,,,,,,,,,,,,,,,
45780,2019-2020,div20,2.15,3.52,3.18,6.0,0.0,2.0,2.0,2.0,...,4.272,5.74,5.785,5.273,5.405,1.5,3.0,1.0,0.0,0.0
45781,2019-2020,div20,2.00,3.60,3.47,7.0,1.0,1.0,3.0,3.0,...,4.028,2.68,4.260,3.693,4.812,1.0,2.0,0.0,0.0,1.0
45782,2019-2020,div20,1.40,4.75,7.02,9.0,1.0,2.0,3.0,3.0,...,3.828,2.06,1.850,1.847,1.882,0.5,0.0,0.0,1.0,0.0
45783,2019-2020,div20,1.43,4.65,6.50,11.0,2.0,2.0,3.0,4.0,...,3.995,1.84,1.795,1.737,2.463,1.2,0.0,1.0,0.0,0.0
45784,2019-2020,div20,2.72,3.51,2.41,13.0,0.0,4.0,6.0,6.0,...,5.222,5.15,8.865,6.443,6.888,0.6,2.4,0.0,1.0,0.0


In [3]:
def base_model(df,target_col):
    #data prep
    features_cols = [x for x in df.columns if x not in ['home_win_flag', 'draw_flag', 'away_win_flag', 'div','season']]
    features_cols = [x for x in features_cols if x.find('odds') == -1]
    #features_col = ['odds_home_team_win',  'odds_away_team_win']
    if target_col == 'draw_flag':
        random_state = 12
    elif target_col == 'home_win_flag': 
        random_state = 456
    else:
        random_state = 4562
        
    random_state = 1
    X_train, X_test, y_train, y_test = train_test_split(df[features_cols], df[target_col], test_size=0.3, random_state=random_state)
       
    
    if target_col != 'draw_flag':
        df_without_draw = df[df.draw_flag==0]
        
    else:
        df_without_draw = df

    # Split data into train and test sets
    X_train, X_test__, y_train, y_test__ = train_test_split(df_without_draw[features_cols], df_without_draw[target_col], test_size=0.3, random_state=random_state)
    #Pipeline
    pipeline = make_pipeline(StandardScaler(), LogisticRegression(penalty='l2', solver='liblinear', max_iter=10000))
    # Fit pipeline to training set and evaluate on test set
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict_proba(X_test)[:, 1]

    # Calculate false positive rate, true positive rate, and area under curve (AUC)
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(fpr, tpr)
    print(f" { target_col } auc is: {roc_auc}.")
    #pipeline.named_steps['logisticregression']
    return pipeline

In [4]:
def create_target_columns_to_classification(df):
    target_cols = ['home_win_flag', 'draw_flag','away_win_flag']
    target_cols_dict = {'home_win_flag':0,'draw_flag':1,'away_win_flag':2}
    def get_y(row):
        for col in target_cols:
            if row[col] == 1:

                y = target_cols_dict[col]
        return y
    df['y'] = df[target_cols].apply(get_y,axis=1)
    df['y'] = df['y'].astype('category')
    return df

In [5]:
p_home_win = base_model(df,'home_win_flag')
p_draw = base_model(df,'draw_flag')
p_away_win = base_model(df,'away_win_flag')

 home_win_flag auc is: 0.6688590396788637.
 draw_flag auc is: 0.5342817267872827.
 away_win_flag auc is: 0.6684308366764647.


In [41]:
target_col = 'home_win_flag'
features_cols = [x for x in df.columns if x not in ['home_win_flag', 'draw_flag', 'away_win_flag', 'div','season']]
features_cols = [x for x in features_cols if x.find('odds') == -1]
target_cols = ['home_win_flag','draw_flag','away_win_flag']

#features_col = ['odds_home_team_win',  'odds_away_team_win']
X_train, X_test, y_train, y_test = train_test_split(df[features_cols], df[target_col], test_size=0.3, random_state=23)
print(f'train data length is {len(y_train)}')
print(f'test data length is {len(y_test)}')
val_df = pd.DataFrame(y_test)
val_df.columns = ['home_win_flag']
val_df['draw_flag'] = df.loc[y_test.index.values]['draw_flag']
val_df['away_win_flag'] = df.loc[y_test.index.values]['away_win_flag']
val_df['home_win_flag_base_pred'] = p_home_win.predict(X_test)
val_df['draw_flag_base_pred'] = p_draw.predict(X_test)
val_df['away_win_flag_base_pred'] = p_away_win.predict(X_test)
val_df = create_target_columns_to_classification(val_df)
val_df[['home_win_prob_0','home_win_prob_1']] = p_home_win.predict_proba(X_test)
val_df[['draw_prob_0','draw_prob_1']] = p_draw.predict_proba(X_test)
val_df[['away_win_prob_0','away_win_prob_1']] = p_away_win.predict_proba(X_test)
for y in target_cols:    
    fpr, tpr, thresholds = roc_curve(val_df[y],val_df[y+'_base_pred'])
    print(f' {y} auc is: {auc(fpr, tpr)}')

train data length is 13000
test data length is 5572
 home_win_flag auc is: 0.6178853344412132
 draw_flag auc is: 0.5
 away_win_flag auc is: 0.62033914696088


In [42]:
retrain_index = val_df[(val_df.home_win_flag_base_pred != val_df.away_win_flag)].index.values
len(retrain_index)

3679

In [43]:
def create_XGBoost_model(models_output_df_features,target):
    from sklearn.model_selection import GridSearchCV
    # Set XGBoost parameters
    param = {
        'max_depth': [2,4,10],
        'learning_rate': [0.1],
        'subsample': [0.8],
        'colsample_bytree': [0.8],
        'n_estimators': [100, 500]
    }

    # Create a GridSearchCV object and fit it to the training data
    xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=3, n_jobs=-1)
    grid_search = GridSearchCV(xgb_model, param_grid=param, cv=3, verbose=3, n_jobs=-1)
    grid_search.fit(models_output_df_features, target)

    # Print the best parameters and the corresponding score
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best score: {grid_search.best_score_}")

    # Predict on test data with the best estimator from grid search
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(models_output_df_features)
    y_test = target
    # Calculate accuracy
    accuracy = sum(y_test == y_pred) / len(y_test)
    print(f"Accuracy: {accuracy}")
    return best_model

In [44]:
target_col = 'home_win_flag'
features_cols = [x for x in df.columns if x not in ['home_win_flag', 'draw_flag', 'away_win_flag', 'div','season']]
features_cols = [x for x in features_cols if x.find('odds') == -1]

retrain_df = df.loc[retrain_index]
print(len(retrain_df))
retrain_df = create_target_columns_to_classification(retrain_df)



3679


In [ ]:
xgb_m = create_XGBoost_model(retrain_df[features_cols],retrain_df['y'])

In [11]:
val_df.loc[retrain_index,'final_pred']  = xgb_m.predict(retrain_df[features_cols])
val_df

,home_win_flag,draw_flag,away_win_flag,home_win_flag_base_pred,draw_flag_base_pred,away_win_flag_base_pred,y,home_win_prob_0,home_win_prob_1,draw_prob_0,draw_prob_1,away_win_prob_0,away_win_prob_1,final_pred
match_id,,,,,,,,,,,,,,
63521,0.0,0.0,1.0,0.0,0.0,1.0,2,0.518400,0.481600,0.805732,0.194268,0.481600,0.518400,2.0
53550,0.0,1.0,0.0,0.0,0.0,1.0,1,0.531613,0.468387,0.669008,0.330992,0.468387,0.531613,NaN
62500,0.0,0.0,1.0,1.0,0.0,0.0,2,0.373438,0.626562,0.659149,0.340851,0.626568,0.373432,NaN
58166,0.0,0.0,1.0,0.0,0.0,1.0,2,0.895034,0.104966,0.787929,0.212071,0.104965,0.895035,2.0
62306,0.0,1.0,0.0,0.0,0.0,1.0,1,0.778425,0.221575,0.784842,0.215158,0.221571,0.778429,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58867,1.0,0.0,0.0,1.0,0.0,0.0,0,0.113998,0.886002,0.853020,0.146980,0.886004,0.113996,0.0
61686,0.0,0.0,1.0,1.0,0.0,0.0,2,0.122983,0.877017,0.749083,0.250917,0.877017,0.122983,NaN
62176,1.0,0.0,0.0,0.0,0.0,1.0,0,0.821707,0.178293,0.723286,0.276714,0.178295,0.821705,NaN


In [15]:
val_df['final_pred'] = np.where(pd.isnull(val_df['final_pred']), val_df['y'], val_df['final_pred'],)

In [45]:
# Ok, itt tultanitom 

In [17]:
def get_pred_targets(x,colname):
    value = target_cols_dict[colname]
    a = 0
    if value == x:
        a = 1
    
    return a
target_cols_dict = {'home_win_flag':0,'draw_flag':1,'away_win_flag':2}
target_cols = ['home_win_flag','draw_flag','away_win_flag']
for t in target_cols:
    val_df[t+'_pred'] = val_df['final_pred'].apply(lambda x: get_pred_targets(x,t))
for y in target_cols:    
    fpr, tpr, thresholds = roc_curve(val_df[y],val_df[y+'_pred'])
    print(f' {y} auc is: {auc(fpr, tpr)}')

 home_win_flag auc is: 1.0
 draw_flag auc is: 1.0
 away_win_flag auc is: 1.0


In [18]:
#lets validate 

In [25]:
df = pd.read_csv('competition_table.csv',sep=';',index_col=0)
# replace commas for dots and convert previous string column into float
def processCol(col):
    try:
        return col.astype(str).apply(lambda val: val.replace(',','.')).astype(float)
    except:
        return col


df = df.apply(processCol)
#keep_seasons = ['2019-2020','2020-2021','2021-2022']
#df = df[df.season.isin(keep_seasons)]


df.head()

target_col = 'home_win_flag'
features_cols = [x for x in df.columns if x not in ['home_win_flag', 'draw_flag', 'away_win_flag', 'div','season']]
features_cols = [x for x in features_cols if x.find('odds') == -1]
target_cols = ['home_win_flag','draw_flag','away_win_flag']

#features_col = ['odds_home_team_win',  'odds_away_team_win']
X_train, X_test, y_train, y_test = train_test_split(df[features_cols], df[target_col], test_size=0.3, random_state=2222)
val_df = pd.DataFrame(y_test)
val_df.columns = ['home_win_flag']
val_df['draw_flag'] = df.loc[y_test.index.values]['draw_flag']
val_df['away_win_flag'] = df.loc[y_test.index.values]['away_win_flag']
val_df['home_win_flag_base_pred'] = p_home_win.predict(X_test)
val_df['draw_flag_base_pred'] = p_draw.predict(X_test)
val_df['away_win_flag_base_pred'] = p_away_win.predict(X_test)
val_df = create_target_columns_to_classification(val_df)
retrain_index = val_df[(val_df.home_win_flag_base_pred != val_df.away_win_flag)].index.values
val_df.loc[retrain_index,'final_pred']  = xgb_m.predict(X_test.loc[retrain_index][features_cols])
val_df['final_pred'] = np.where(pd.isnull(val_df['final_pred']), val_df['y'], val_df['final_pred'],)
for t in target_cols:
    val_df[t+'_pred'] = val_df['final_pred'].apply(lambda x: get_pred_targets(x,t))
for y in target_cols:    
    fpr, tpr, thresholds = roc_curve(val_df[y],val_df[y+'_pred'])
    print(f' {y} auc is: {auc(fpr, tpr)}')

 home_win_flag auc is: 0.8255795573710789
 draw_flag auc is: 0.700788158368163
 away_win_flag auc is: 0.9588457466899823


In [29]:
import joblib
import pickle

In [33]:
# save model
pickle.dump(p_home_win, open('pipe_home_win_base_model_2.p', "wb"))
pickle.dump(p_draw, open('pipe_p_draw_base_model_2.p', "wb"))
pickle.dump(p_away_win, open('pipe_p_away_win_base_model_2.p', "wb"))
pickle.dump(xgb_m, open('xgb_m_2.p', "wb"))